In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('catELMo/TCRrepertoires.csv')

In [3]:
df2 = pd.read_csv('BindingAffinityPrediction/TCREpitopePairs.csv')

In [4]:
import numpy as np
from sklearn.manifold import TSNE

In [5]:
import pandas as pd
from pathlib import Path
import torch

In [6]:
from allennlp.modules.elmo import Elmo, batch_to_ids

In [7]:
from allennlp.modules.token_embedders import ElmoTokenEmbedder

In [8]:
model_dir = Path('catELMo_model')
weights = model_dir/'weights.hdf5'
options = model_dir/'options.json'


In [9]:
embedder  = Elmo(options,weights,2) #num_output_representations = 2  # cuda_device=-1 for CPU

In [10]:
def catELMo_embedding(x):
    # print(embedder(batch_to_ids(list(x)))['elmo_representations'][0].sum(dim = 0).mean(dim = 0).tolist())
    return np.array(embedder(batch_to_ids(list(x)))['elmo_representations'][0].sum(dim = 0).mean(dim = 0).tolist())


In [11]:
dat = df2
dat['tcr_embeds'] = None
dat['epi_embeds'] = None

# dat['epi_embeds'] = dat[['epi']].applymap(lambda x: catELMo_embedding(x))['epi']
# dat['tcr_embeds'] = dat[['tcr']].applymap(lambda x: catELMo_embedding(x))['tcr']


In [12]:
num_elements = 5000

In [13]:
# dat['epi_embeds'] = dat[['epi']][:1000].applymap(lambda x: catELMo_embedding(x))['epi']

In [ ]:
dat['tcr_embeds'] = dat[['tcr']][:num_elements].applymap(lambda x: catELMo_embedding(x))['tcr']

In [ ]:
'hi'

In [ ]:
dat.iloc[num_elements - 1]

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components = 50)

In [ ]:
tcr_embeds_np = np.array(dat['tcr_embeds'][:num_elements].values.tolist())

In [ ]:
pca.fit(tcr_embeds_np)

In [ ]:
transformed = pca.transform(tcr_embeds_np)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tcr_embedded = TSNE(n_components=2, learning_rate='auto', init='pca').fit_transform(transformed)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df2

In [ ]:
colors = ['orange']*num_elements
for i in range(num_elements):
    if df2.iloc[i].binding == 1:
        colors[i] = 'red'
plt.scatter(tcr_embedded[:, 0], tcr_embedded[:, 1], color = colors)